## Captura de datos y entrega de acciones

In [45]:
import requests
from bs4 import BeautifulSoup

def agente_web(url="https://www.sura.co/seguros"):
    resp = requests.get(url)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "html.parser")

    acciones = []

    # Buscar dentro de <nav> y <main> solamente
    secciones = soup.find_all(["nav", "main", "header", "section"])
    for seccion in secciones:
        for a in seccion.find_all("a", href=True):
            texto = a.get_text(strip=True)
            if texto and len(texto) > 3 and not any(x in a["href"] for x in ["facebook", "twitter", "legal", "aviso"]):
                acciones.append({
                    "tipo": "enlace",
                    "texto": texto,
                    "href": a["href"]
                })

        for b in seccion.find_all("button"):
            texto = b.get_text(strip=True)
            if texto and len(texto) > 3:
                acciones.append({
                    "tipo": "botón",
                    "texto": texto
                })

    return acciones

# Ejemplo de uso
if __name__ == "__main__":
    acciones = agente_web()
    print(f"Se encontraron {len(acciones)} acciones posibles:\n")
    for act in acciones[:20]:
        print(act)

Se encontraron 1119 acciones posibles:

{'tipo': 'enlace', 'texto': 'Skip to Main Content', 'href': '#main-content'}
{'tipo': 'enlace', 'texto': 'Ley de transparencia', 'href': 'https://www.segurossura.com.co/paginas/ley-de-transparencia.aspx'}
{'tipo': 'enlace', 'texto': 'Iniciar sesión', 'href': '#'}
{'tipo': 'enlace', 'texto': 'Sucursal Virtual Personas', 'href': 'https://login.sura.com/sso/servicelogin.aspx?continueTo=https%3A%2F%2Fsucursal.segurossura.com.co&service=clienteseguros'}
{'tipo': 'enlace', 'texto': 'Solicita tu clave', 'href': 'https://www.segurossura.com.co/paginas/registro-de-usuarios.aspx'}
{'tipo': 'enlace', 'texto': 'Proveedores', 'href': 'https://proveedor.sura.com/Paginas/Publicas/Home.aspx'}
{'tipo': 'enlace', 'texto': 'Asesores', 'href': 'https://login.sura.com/sso/servicelogin.aspx?continueTo=https%3A%2F%2Fasesores.segurossura.com.co&service=portaluasesores'}
{'tipo': 'enlace', 'texto': 'Personas', 'href': '#'}
{'tipo': 'enlace', 'texto': 'Movilidad', 'href':

In [46]:
# --- buscador_local.py ---
from difflib import SequenceMatcher

def buscar_acciones_relacionadas(pregunta, acciones, top_n=5):
    pregunta = pregunta.lower()
    resultados = []

    for a in acciones:
        texto = a["texto"].lower()
        similitud = SequenceMatcher(None, pregunta, texto).ratio()
        if any(palabra in texto for palabra in pregunta.split()):
            similitud += 0.2  # bonus si hay palabras exactas
        resultados.append((similitud, a))

    resultados.sort(key=lambda x: x[0], reverse=True)
    return [r[1] for r in resultados[:top_n]]

In [48]:
acciones = agente_web("https://www.sura.co/seguros")
print(f" Acciones detectadas: {len(acciones)}")

pregunta = input("¿Qué necesitas hacer en la web?")

acciones_relacionadas = buscar_acciones_relacionadas(pregunta, acciones, top_n=5)

if acciones_relacionadas:
    print(" Acciones relacionadas:")
    for i, a in enumerate(acciones_relacionadas, 1):
        tipo = "Enlace" if a["tipo"] == "enlace" else "Botón"
        link = f"{a['href']}" if "href" in a else ""
        print(f"{i}. {tipo}: {a['texto']} {link}")
else:
    print(" No se encontraron acciones relacionadas.")

 Acciones detectadas: 1119
 Acciones relacionadas:
1. Enlace: Seguro de Motos https://www.sura.co/seguros/personas/movilidad/motos
2. Enlace: Seguro de Motos https://www.sura.co/seguros/personas/movilidad/motos
3. Enlace: Seguro de Motos https://www.sura.co/seguros/personas/movilidad/motos
4. Enlace: Seguro de Motos https://www.sura.co/seguros/personas/movilidad/motos
5. Enlace: Seguro de Motos https://www.sura.co/seguros/personas/movilidad/motos
